In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold

In [2]:
TRAIN = [#'../input/rapids-ensemble-trends-both-oof/rapids_svr_train_preds_out.csv',
        #'../input/rapids-ensemble-trends-both-oof/rapids_ridge_train_preds_out.csv',
        '../input/rapids-ensemble-trends-oof/rapids_ensemble_train_preds_out.csv',
        #'../input/trends-tabular-nn-0-159/mix_train_preds_out.csv',
        #'../input/trends-tabular-nn-0-159/mse_train_preds_out.csv',
        '../input/trends-tabular-nn-0-159/blended_train_preds_out.csv',
        #'../input/trends-gcn-tab-oof/gcn_train_preds_out.csv',
        '../input/trends-transformers-tabular-learner-oof/transformer_train_preds_out.csv',
        '../input/trends-gcn-248-tab-oof/gcn_train_preds_out.csv',       
        ]

In [3]:
TEST = [#'../input/rapids-ensemble-trends-both-oof/rapids_svr_test_preds_out.csv',
        #'../input/rapids-ensemble-trends-both-oof/rapids_ridge_test_preds_out.csv',
        '../input/rapids-ensemble-trends-oof/rapids_ensemble_test_preds_out.csv',
        #'../input/trends-tabular-nn-0-159/mix_test_preds_out.csv',
        #'../input/trends-tabular-nn-0-159/mse_test_preds_out.csv',
        '../input/trends-tabular-nn-0-159/blended_test_preds_out.csv',
        #'../input/trends-gcn-tab-oof/gcn_test_preds_out.csv',
        '../input/trends-transformers-tabular-learner-oof/gcn_test_preds_out.csv',
        '../input/trends-gcn-248-tab-oof/gcn_test_preds_out.csv',
        ]

In [4]:
#display(pd.read_csv(TRAIN[0]).head())
#display(pd.read_csv(TRAIN[1]).head())
#display(pd.read_csv(TRAIN[2]).head())
#display(pd.read_csv(TRAIN[3]).head())
#display(pd.read_csv(TRAIN[4]).head())
#display(pd.read_csv(TRAIN[5]).head())
#display(pd.read_csv(TRAIN[6]).head())

In [5]:
train_scores = pd.read_csv('../input/trends-assessment-prediction/train_scores.csv')
train_scores.head()



,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,66.532630,NaN,NaN,52.108977,69.993075
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421


In [6]:
# prepare for STRATIFIED KFOLD
y_data = train_scores.copy()
y_data['bin_age_7'] = pd.qcut(y_data['age'].rank(method='first'), q=7, labels=False)
kf_split=10
kf = StratifiedKFold(n_splits=kf_split, shuffle=True, random_state=2020) # n_repeats=10

In [7]:
targets = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']

In [8]:
# Don't do fillna(mean), rather ignore NaN entry while training
#train_scores = train_scores.fillna(train_scores.mean())

In [9]:
from sklearn.linear_model import LinearRegression, ElasticNetCV,BayesianRidge,HuberRegressor, Ridge
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [10]:
def get_preds(var, all_targets_in_train=False):
    
    if all_targets_in_train:
        features = targets
    else:
        features = [var]
    
    X_train = pd.DataFrame()#[]
    X_train_incl_nan = pd.DataFrame()#[]
    y_train = []
    X_test = pd.DataFrame()#[]
    
    ## remove NaN entries from training
    y_train = train_scores[train_scores[var].notnull()][var]
    y_idx_to_keep = y_train.index
    y_train = y_train.values
    
    c=0
    for f in TRAIN:
        df = pd.read_csv(f)
        
        ## filled 0 prediction won't be evaluated when calling the metric
        #X_train_incl_nan.append(df[features].copy().fillna(0)) 
        X_train_incl_nan = pd.concat([X_train_incl_nan, df[features].copy().fillna(0)],axis=1)
        
        df = df[features].iloc[y_idx_to_keep]
        #X_train.append(df)
        X_train = pd.concat([X_train, df],axis=1)
        
        c+=1
    
    X_train = np.array(X_train) #.T
    X_train_incl_nan = np.array(X_train_incl_nan) #.T
    
    c=0
    for f in TEST:
        df = pd.read_csv(f)
        #X_test.append(df[features])
        X_test = pd.concat([X_test, df[features]],axis=1)
        c+=1
        
    X_test = np.array(X_test) #.T
    
    N = len(X_test)
    
    strat_y_df = y_data['bin_age_7'].iloc[y_idx_to_keep]
    
    #LinearRegression()
    #model = ElasticNetCV(l1_ratio=0.5, cv = kf.split(X_train, strat_y_df.values))
    #model = BayesianRidge()
    
    preds_test=np.zeros(X_test.shape[0])
    preds_val=np.zeros(X_train_incl_nan.shape[0])
    
    print(var)
    fld_cn=0
    """for train_idx, val_idx in kf.split(X_train, strat_y_df.values):
        #print('fold',fld_cn)
        fld_cn+=1
        #val_data = X_train_incl_nan[val_idx]
        val_data = X_train[val_idx]
        
        model=LinearRegression()#HuberRegressor(max_iter=10000) 
        model.fit(X_train[train_idx], y_train[train_idx])
        
        
        y_true_mod = y_train[train_idx].copy()
        #noise_range = 0.15 #torch.tensor([0.145, 0.151, 0.151, .181, .177])
        #noise = np.random.rand(y_true_mod.shape[0])*(np.sign(np.random.rand(y_true_mod.shape[0])-0.5))*2*noise_range+1
        #y_true_mod += noise
        ##print(y_true_mod.shape,'##',noise.shape, y_true_mod.shape)

        lr= 0.05
        n_est=10000
        #model =  XGBRegressor(n_estimators=n_est, learning_rate=lr, random_state=2020, max_deth=3, gamma = 1, subsample=0.8)
        #model.fit(X_train[train_idx], y_true_mod,#y_train[train_idx], #
        #          early_stopping_rounds=10, eval_set=[(val_data, y_train[val_idx])], verbose=400)#, eval_metric = 'eval_score')
        
        #print('Best iteration: '+ str(model.best_iteration), 'Best ntree_limit: '+ str(model.best_ntree_limit), 'Best score:', str(model.best_score)) #model.best_score,


        #model=lgb.LGBMRegressor(max_depth= 9, learning_rate= lr, num_leaves= 128,  n_estimators= n_est, n_jobs= -1)
        #model = model.fit(X_train[train_idx], y_true_mod, categorical_feature='auto',
        #            eval_set=[(val_data, y_train[val_idx])], eval_metric='eval_score',
        #            verbose=400, early_stopping_rounds=20)

        preds_test+=model.predict(X_test)
        #preds_val+=model.predict(X_train_incl_nan)
        preds_val[val_idx] = model.predict(val_data)
        #print(preds_val[val_idx][:10], y_train[val_idx][:10],val_idx)
        
    return preds_val   , preds_test / kf_split #preds_val/ kf_split 
    """
    
    model = HuberRegressor(max_iter=10000) 
    model.fit(X_train, y_train)
    #print(f'{var}  --> {model.coef_}')
    
    #lr= 0.2
    #n_est=20000
    #dtrain=xgb.DMatrix(X_train, y_train)
    #model = XGBRegressor(n_estimators=n_est, learning_rate=lr, random_state=2019, max_deth=2) #10)
    #xgb_params = {'max_deth':10}
    #model=xgb.cv(xgb_params, dtrain, num_boost_round=10000, nfold=10, seed=2020, stratified=False,
    #                 early_stopping_rounds=40, verbose_eval=400, show_stdv=True)
    
    if var=='age':
        pp = 1.01
    else:
        pp = 1
    
    return model.predict(X_train_incl_nan)*pp, model.predict(X_test)*pp

In [11]:
Y1 = np.zeros((5877, 5))
Y2 = np.zeros((5877, 5))

In [12]:
def eval_score(y_true, y_pred):
    score = 0
    w = [0.3, 0.175, 0.175, 0.175, 0.175]
    
    y_pred = y_pred * np.abs(np.sign(y_true))
    #print(y_pred[:6])
    #print(y_true[:6])
    for i in range(5):
        t = np.mean(np.sum(np.abs(y_true[:,i] - y_pred[:,i]), axis=0)/np.sum(y_true[:,i], axis=0))
        score+= w[i] * t
    return score

In [13]:
for i,t in enumerate(targets):
    preds = get_preds(t)
    Y1[:, i] = preds[0]
    Y2[:, i] = preds[1]

print(f'\neval_score (only target feature in) --> {eval_score(train_scores[targets].fillna(0).values, Y1)}')

age
domain1_var1
domain1_var2
domain2_var1
domain2_var2

eval_score (only target feature in) --> 0.15747079654465518


In [14]:
# 10 over xgb
#0.14270763803463116
# loo
#0.19012104360594778

#huber over
#0.1572850316918
#loo
#0.18803949261191064

#baseline:
#age  --> [ 0.60537775  0.04388886  0.24512624  0.18929293 -0.07399312]
#domain1_var1  --> [ 0.45066269 -0.01907722  0.30174618  0.31235954  0.0324813 ]
#domain1_var2  --> [ 0.26364107  0.17387063  0.35058835  0.23846849 -0.02064332]
#domain2_var1  --> [0.37943906 0.11920945 0.29557785 0.18110915 0.14764214]
#domain2_var2  --> [0.32113812 0.06524198 0.37241127 0.25134411 0.04839565]

#eval_score (only target feature in) --> 0.15728312497438668

In [15]:
for i,t in enumerate(targets):
    preds = get_preds(t, all_targets_in_train = True)
    Y1[:, i] = preds[0]#+.1
    Y2[:, i] = preds[1]

print(f'\neval_score (all targets as input features) --> {eval_score(train_scores[targets].fillna(0).values, Y1)}')

age
domain1_var1
domain1_var2
domain2_var1
domain2_var2

eval_score (all targets as input features) --> 0.15711953942196563


In [16]:
train_scores.head()

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,66.532630,NaN,NaN,52.108977,69.993075
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421


In [17]:
#10 over xgb
#0.13338125596050335
#loo
#0.19060644149447542

#huber over
#0.1569486106860847
#loo
#0.18857025160340465

#Baseline
#eval_score (all targets as input features) --> 0.15694902489728832

In [18]:
#test_df =pd.read_csv("/kaggle/input/trends-train-test-creator/test.csv")

test_df =pd.read_csv(TEST[0])

sub_df = pd.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]],
                 id_vars=["Id"], 
                 value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df['Predicted'] = Y2.flatten()
sub_df.head(20)

,Id,Predicted
0,10003_age,57.719742
5877,10003_domain1_var1,51.722623
11754,10003_domain1_var2,59.659334
17631,10003_domain2_var1,49.980102
23508,10003_domain2_var2,56.139408
1,10006_age,64.543479
5878,10006_domain1_var1,55.808793
11755,10006_domain1_var2,59.884761
17632,10006_domain2_var1,50.408135
23509,10006_domain2_var2,52.554521


In [19]:
sub_df.to_csv("submission.csv", index=False)